In [1]:
import os
import re
import pandas as pd
from ast import literal_eval
from difflib import SequenceMatcher
import numpy as np

df_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#9_tf_idf'}.xlsx")
df = pd.read_excel(df_name, index_col=0)

In [2]:
cases = list(df["case"])    
cases = [item for sublist in cases for item in literal_eval(sublist)]
cases = list(set(cases))
cases.sort()
result = pd.DataFrame(0, index=np.arange(len(df)), columns = cases)
result['cid'] = df['cid']
cols = result.columns.tolist()
cols = cols[-1:] + cols[0:-1]
result = result[cols]

In [3]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

for i in range(len(df['case'])):
    
    cs = literal_eval(df.loc[i, 'case'])
    for c in cs:
        for j in range(len(cases)):
            if similar(c, cases[j]) > 0.8:# 차이 있나 확인해보기
                result.at[i, cases[j]] = 1
                
x2name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#10_case'}.xlsx")
result.to_excel(x2name)

In [4]:
laws = list(df["law"])    
laws = [item for sublist in laws for item in literal_eval(sublist)]
laws = list(set(laws))
laws.sort()
result = pd.DataFrame(0, index=np.arange(len(df)), columns = laws)
result['cid'] = df['cid']
cols = result.columns.tolist()
cols = cols[-1:] + cols[0:-1]
result = result[cols]
len(laws)

2238

In [5]:
for i in range(len(df['law'])):
    
    lw = literal_eval(df.loc[i, 'law'])
    for l in lw:
        for j in range(len(laws)):
            if l == laws[j]:
                result.at[i, laws[j]] = 1

In [6]:
count = 0
new_law = list()
for i in range(len(laws)):
    if len(result[result[laws[i]] == 1]) < 3: # 차이 있나 확인해보기
        count+=1
    else:
        new_law.append(laws[i])
print(count)

1576


In [7]:
result = pd.DataFrame(0, index=np.arange(len(df)), columns = new_law)
result['cid'] = df['cid']
cols = result.columns.tolist()
cols = cols[-1:] + cols[0:-1]
result = result[cols]


for i in range(len(df['law'])):
    
    lw = literal_eval(df.loc[i, 'law'])
    for l in lw:
        for j in range(len(new_law)):
            if l == new_law[j]:
                result.at[i, new_law[j]] = 1
                
x2name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#10_law'}.xlsx")
result.to_excel(x2name)

In [8]:
len(new_law), len(cases)

(662, 687)

In [9]:
df = df.drop(columns = ['case', 'law'])
x2name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#10_final'}.xlsx")
df.to_excel(x2name)